<div align="center"><img src="./images/DLI_Header.png"></div>

# Final Exercise

Let's test your NVSHMEM skills! In this notebook We provide a fully-implemented CUDA application that works on a single GPU, and your job is to convert it to work correctly in NVSHMEM for an arbitrary number of PEs.

## Objectives

By the time you complete this notebook you will:

- Demonstrate your ability to write NVSHMEM code for an arbitrary number of PEs.

## 1D Wave Equation

As with the Jacobi program, you don't need to really understand this algorithm to work with the code, but, we will take time here to introduce it.

The problem we'll solve is the 1D wave equation:

$$
\frac{\partial^2}{\partial t^2} u(x,t) = c^2 \frac{\partial^2}{\partial x^2} u(x,t)
$$

Here $u = u(x,t)$ is a scalar function of space ($x$) and time ($t$), and $c$ is a (constant) characteristic wave speed (for example, the speed of sound if the wave in question is propagating in air).

Implementing the centered-difference discretization of the spatial and time derivatives, one way to write this is:

$$
\frac{1}{\Delta t^2} \left(u_{i}^{n+1} - 2 u_{i}^{n} + u_{i}^{n-1}\right) = \frac{c^2}{\Delta x^2} \left(u_{i+1}^{n} - 2 u_{i}^{n} + u_{i-1}^{n}\right)
$$

Where subscripts denote spatial indices and superscripts denote timesteps. Rearranging this for the unknown $u_{i}^{n+1}$ in terms of known quantities at timesteps $n$ and $n-1$, we have:

$$
u_{i}^{n+1} = 2 u_{i}^{n} - u_{i}^{n-1} + \left(\frac{c\, \Delta t}{\Delta x}\right)^2 \left(u_{i+1}^{n} - 2 u_{i}^{n} + u_{i-1}^{n}\right)
$$

### Solving

To solve using this method, we simply need to retain the value of the solution at two previous timesteps, and then replace the old data after each update.

We're going to specify a 1D domain from $x = 0.0$ to $x = 1.0$, and discretize into $N$ points with a grid spacing of $\Delta x = 1.0\, /\, (N - 1)$. $\Delta t$ [must be chosen](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition) so that it is less than or equal to $c \Delta x$. To simplify, we'll choose $c = 1.0$ so that we don't have to worry about that term floating around.

We're going to specify that $u(0, t) = u(1, t) = 0$. We can think of this like we're solving a wave propagating in a string, where the two ends of the string are held taut. What we specify is the initial condition $u(x, 0)$, a simple sine wave, as well as an initial condition (the velocity at $t = 0$ is zero, which is effectively implemented by starting with $u^{n} == u^{n-1}$).

The period of this wave is 1.0, so after simulating up to $t = 1$, the wave should return exactly to where it started. We will use that to verify our solution -- the check at the end of the code will print an "error" which is the $L^2$ norm of the current solution with respect to the initial solution.

## A Single GPU Implementation

This example is implemented in standard CUDA for a single GPU in [exercises/wave.cpp](exercises/wave.cpp). Take some time to review the algorithm and its parallel implementation, and examine the output below.

In [ ]:
!nvcc -x cu -arch=sm_70 -o wave exercises/wave.cpp
!./wave

Before you make any changes, save the current file in case you want to revisit it:

In [ ]:
!cp exercises/wave.cpp exercises/wave_original.cpp

## Assessment: Implement with NVSHMEM

Now implement this with NVSHMEM, dividing the domain equally into $M$   subdomains (for $M$   PEs), with PE 0 owning $x = [0, 1 / M]$  , PE 0 owning $x = [1/M, 2/M]$  , etc. Keep the points at $x = 0$   and $x = 1$   completely fixed.

Some things to keep in mind as you're implementing your solution:
- Currently the initialization routine assumes the full domain is available. You'll have to modify that so that each PE sets the appropriate initial conditions for its location in the spatial domain.
- Similarly, for the solution check, make sure you do this properly across all PEs by summing up the L2 norm locally and then reducing all over PEs.
- You'll have to do some point-to-point communication inside the actual `wave_update()` routine to get the "halo" data from neighboring PEs (but be careful not to update the boundary points, one of which lives on PE 0, the other on PE $M$-1).
- It's OK if your solution is not any faster than the single-GPU case. We are mostly focusing on writing correct NVSHMEM code in this lab. Given the default value of `NUM_POINTS`, the amount of work per kernel is small enough that we're not really efficiently utilizing the GPU. If you want to do a more realistic performance comparison, set `NUM_POINTS` to a much larger number (but then cap the integration to, say, 10000 steps so that it completes in a reasonable amount of time). You can use the Jupyter notebook `%time` magic function to easily compare application runtime (e.g. `%time !nvshmrun -np $NUM_DEVICES ...`).

### Difficulty Level: Hard

For those of you who want to get the most learning out of this exercise, and are up for a challenge, we highly recommend simply refactoring [exercises/wave.cpp](exercises/wave.cpp) as it currently exists, looking back over previous notebooks as needed to complete your work.

### Difficulty Level: Medium

If you need some help figuring out what to do, but still want to figure most of the details out for yourself, run the following cell to copy a version of the wave program into `wave.cpp` that is commented with a lot of `FIXMEs` to help guide your work:

In [ ]:
!cp exercises/wave_fixmes.cpp exercises/wave.cpp

### Difficulty Level: Easier

If you are really struggling, you can execute the following cell to copy a near-solution implementation into `wave.cpp` that will still require you to address some `FIXMEs` to complete the assessment.

**NOTE:** Because this exercise is used as an assessment for the course, we will not be providing a full solution implementation.

In [ ]:
!cp exercises/wave_fixmes_easy.cpp exercises/wave.cpp

### Run the Code

Use the following cell to compile and run your program. Once your code has been refactored to successfully use NVSHMEM while continuing to produce the correct results, continue below to submit it for assessment.

**NOTE:** If you try to run the following without having initialized NVSHEM in your code, it will hang indefinitely, and you will need to use the *Kernel* menu above to interupt the kernel.

In [ ]:
!nvcc -x cu -arch=sm_70 -rdc=true -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o wave exercises/wave.cpp
!nvshmrun -np $NUM_DEVICES ./wave

### Run the Assessment

If your code is ready to be assessed, execute the following cell:

In [ ]:
!/dli/assessment/run_assessment.sh

### Get Credit for Your Work

After successfully passing the assessment above, revisit the webpage where you launched this interactive environment and click on the **"ASSESS TASK"** button as shown in the screenshot below. Doing so will give you credit for this part of the workshop that counts towards earning a **certificate of competency** for the entire course.

![Run the assessment](images/run_the_assessment.png)

## Summary

Congratulations! You've now mastered the fundamentals of NVSHMEM and are ready to begin applying it in your own problems.